# Wild Wolf Watch Observations

> Here’s a link to [our] project: https://www.zooniverse.org/projects/mlsmax/wild-wolf-watch, which focuses on classifying trap-camera photos from Eastern NC in order to get a better picture of the biological diversity after the reintroduction of critically endangered Red Wolves into the region (there’s a lot more you can read about the project on the site as well).

> Basically, we’re running into trouble with the data exports from the website. Some of the columns have nested JSON that I would like to have unnested and use for data analysis.

> I’ve gone ahead and attached an excel file of what our data exports right now look like. Essentially, what we’re looking for is all nested in the Annotations column.

In this notebook, I'll show how to unpack the `annotations` column in the example dataset into a separate Pandas DataFrame. I'll also demonstrate how to join this new DataFrame with select columns from the original. 

Ultimately, I'd love to see the images rendered inline in the notebook next to the annotations, but I haven't found a way to reference the images using the subject IDs or filename in the dataset. Maybe there's more data available for export? Or a way to look up the image given a subject ID using the Zooniverse API?

## Load the data

In [1]:
import json
import pandas as pd
pd.options.display.max_colwidth = 1024

The data sample is in the same folder as this notebook. I read it into a `pandas.DataFrame`.

In [2]:
raw_df = pd.read_csv('./wild-wolf-watch-classifications.csv')

I need to get a sense of the data structure. Here I pull a sample of 5 rows. I run this notebook cell multiple times and note how the values different across rows.

In [81]:
raw_df.sample(5)

,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
3228,78455663,not-logged-in-15759e250e28f051a923,NaN,15759e250e28f051a923,4466,Classify Motion-Activated Camera Pictures,47.58,2017-11-14 16:35:28 UTC,NaN,NaN,"{""session"":""7e801f73cb80b5c7b800c2f2a5006132ca9cd53c01b036d7f03188e01debd6c4"",""viewport"":{""width"":1024,""height"":672},""started_at"":""2017-11-14T16:35:19.881Z"",""user_agent"":""Mozilla/5.0 (iPad; CPU OS 10_3_3 like Mac OS X) AppleWebKit/603.3.8 (KHTML, like Gecko) Version/10.0 Mobile/14G60 Safari/602.1"",""utc_offset"":""18000"",""finished_at"":""2017-11-14T16:35:28.122Z"",""live_project"":false,""user_language"":""en"",""subject_dimensions"":[{""clientWidth"":433,""clientHeight"":244,""naturalWidth"":1152,""naturalHeight"":648}]}","[{""task"":""T0"",""value"":[{""choice"":""BEAR"",""answers"":{""HOWMANY"":""1"",""WHATBEHAVIORSDOYOUSEE"":[""INTERACTING""],""ARETHEREANYYOUNGPRESENT"":""NO""},""filters"":{}}]}]","{""10710121"":{""retired"":null,""Filename"":""mfdc7677_28926515723_o.jpg""}}",10710121
4071,85802169,BetsyK,1684331.0,999ceeb88c16b8bda6e0,4466,Classify Motion-Activated Camera Pictures,47.58,2018-01-13 17:05:34 UTC,NaN,NaN,"{""session"":""7880b822927769e85a0dd5e9b992f922e62ed4a1dfd5ff497ef8e197ade64f19"",""viewport"":{""width"":678,""height"":483},""started_at"":""2018-01-13T17:04:59.754Z"",""user_agent"":""Mozilla/5.0 (iPad; CPU OS 11_0_2 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A421 Safari/604.1"",""utc_offset"":""18000"",""finished_at"":""2018-01-13T17:05:34.381Z"",""live_project"":false,""user_language"":""en"",""user_group_ids"":[],""subject_dimensions"":[{""clientWidth"":433,""clientHeight"":244,""naturalWidth"":1152,""naturalHeight"":648}]}","[{""task"":""T0"",""value"":[{""choice"":""DEER"",""answers"":{""HOWMANY"":""1"",""WHATBEHAVIORSDOYOUSEE"":[""EATING""],""ARETHEREANYYOUNGPRESENT"":""NO""},""filters"":{}}]}]","{""10722963"":{""retired"":null,""Filename"":""MFDC3589.jpg""}}",10722963
700,70128878,Katy_H,1679909.0,bf882cda7a78bb06f7e7,4466,Classify Motion-Activated Camera Pictures,47.58,2017-09-13 00:19:42 UTC,NaN,NaN,"{""session"":""8b04e40307417a5c1c681c20622abc369f686fa15f49c49e1379645bd2e9304e"",""viewport"":{""width"":414,""height"":696},""started_at"":""2017-09-13T00:19:38.817Z"",""user_agent"":""Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_3 like Mac OS X) AppleWebKit/603.3.8 (KHTML, like Gecko) Version/10.0 Mobile/14G60 Safari/602.1"",""utc_offset"":""14400"",""finished_at"":""2017-09-13T00:19:42.694Z"",""live_project"":false,""user_language"":""en"",""user_group_ids"":[],""subject_dimensions"":[{""clientWidth"":389,""clientHeight"":219,""naturalWidth"":1152,""naturalHeight"":648}]}","[{""task"":""T0"",""value"":[{""choice"":""IMNOTSURE"",""answers"":{},""filters"":{}}]}]","{""10710452"":{""retired"":null,""Filename"":""mfdc8793_34291388883_o.jpg""}}",10710452
1309,73865141,not-logged-in-2d82cb35f742f2da4e42,NaN,2d82cb35f742f2da4e42,4466,Classify Motion-Activated Camera Pictures,47.58,2017-10-10 17:18:59 UTC,NaN,NaN,"{""session"":""a5e5689f301d7dab49e369b1f3e7406003dbf3f9e629b3c0ba21f16c7b3321c3"",""viewport"":{""width"":1024,""height"":704},""started_at"":""2017-10-10T17:18:48.332Z"",""user_agent"":""Mozilla/5.0 (iPad; CPU OS 10_3_3 like Mac OS X) AppleWebKit/603.3.8 (KHTML, like Gecko) Version/10.0 Mobile/14G60 Safari/602.1"",""utc_offset"":""14400"",""finished_at"":""2017-10-10T17:18:59.213Z"",""live_project"":false,""user_language"":""en"",""subject_dimensions"":[{""clientWidth"":433,""clientHeight"":244,""naturalWidth"":1152,""naturalHeight"":648}]}","[{""task"":""T0"",""value"":[{""choice"":""BEAR"",""answers"":{""HOWMANY"":""1"",""WHATBEHAVIORSDOYOUSEE"":[""MOVING""],""ARETHEREANYYOUNGPRESENT"":""NO""},""filters"":{}}]}]","{""10710671"":{""retired"":null,""Filename"":""mfdc9246_34936937572_o.jpg""}}",10710671
27,62756504,mlsmax,16

## Limit to the newest version

The structure of the JSON in the `annotation` column appears to vary greatly with `workflow_version`. I assume this means the setup of the questionnaire on the Zooniverse site has evolved over time. I wonder how many different versions there are?

In [30]:
raw_df.workflow_version.value_counts()

47.58    4484
44.57      89
42.56      14
34.50      12
43.57       6
36.52       4
41.55       2
Name: workflow_version, dtype: int64

The vast majority of the rows are associated with version 47.58. I am going to focus on this version to keep the data transformations simple for now. (I can always come back and deal with the other rows later, if neccessary.)

In [31]:
latest_df = raw_df[raw_df.workflow_version == 47.58].copy()

## Unnest the annotations JSON

The `annotations` column appears to hold the responses from users (i.e., annotators) about subjects (i.e., camera images). The responses are in JSON format with the key information in a `value` list which may have multiple elements. Here's one example, indented and with comments for readability:

```
[
  {
    "task":"T0",                                 # all entries have just one task at the moment
    "value": [                                   # multiple annotations per camera image are possible
      {
        "choice": "BEAR",                        # what the user sees in the picture
        "answers": {                             # follow on questions based the "choice"
          "HOWMANY":"1",                         # note this is a string, not an int
          "WHATBEHAVIORSDOYOUSEE": ["STANDING"], # can have multiple values
          "ARETHEREANYYOUNGPRESENT":"NO"
        },
        "filters": {}
      }
    ]
  }
]
```

I can unnest this column into its own DataFrame. Since there can be multiple annotations per subject (i.e., row in the original table), I need to include a key that can link the annotations DataFrame with the original. The `classification_id` column looks like a reasonable choice. I need to make sure it's unique across all rows first.

In [32]:
latest_df.classification_id.nunique() == len(latest_df)

True

Perfect.

There is a handy function, `pandas.io.json.json_normalize`, that can do the heavy lifting of exploding a list of JSON objects into a DataFrame. I need to do some prep work for it to work properly.

First, I need to decode the JSON strings in the `annotations` column into real Python objects (e.g., lists, dictionaries, etc.) and make sure every row has just one Python dictionary (i.e., a single task). As I noted above, it looks like there's a single task per row at present, so all I need to do here is validate that assumption and fetch the first task list element.

In [33]:
def load_json(value):
    """Decodes a JSON string, asserts there is exactly one element in the decoded
    list, and returns that one element.
    """
    tasks = json.loads(value)
    assert len(tasks) == 1
    return tasks[0]

I apply my `load_json` function to the `annotations` column and overwrite the `annotations` column in the DataFrame with the resulting series. If I get an exception here, I know my assumption about a single task per row is incorrect.

In [34]:
latest_df['annotations'] = latest_df.annotations.apply(load_json)

Here's what I now see in `annotations.

In [35]:
latest_df.annotations.head(1)

127    {'task': 'T0', 'value': [{'choice': 'IMNOTSURE', 'answers': {}, 'filters': {}}]}
Name: annotations, dtype: object

Next, I need to insert the `classification_id` in the `task` dictionary stored in the `annotation` column. I need to do this so because the `json_normalize` function only operates on the content of this column, not the surrounding DataFrame.

In [39]:
def id_annotation(row):
    """Duplicates the row to avoid modifying the original in place, 
    adds the value of the classification_id column to the dictionary
    in the annotations column, and returns the new row.
    """
    # not strictly necessary, but keeps the original row / dataframe untouched
    # immutability is good!
    new_row = row.copy()
    new_row['annotations']['classification_id'] = new_row['classification_id']
    return new_row

I apply my `id_annotation` function to the rows of the DataFrame.

In [37]:
latest_df = latest_df.apply(id_annotation, axis=1)

Now I can use the `json_normalize` function explode the `annotation` dictionaries into their own DataFrame. I pass it the `annotations` column, indicate that the `value` list contains the data of interest, and note that it should retain the `classification_id` as metadata associated with each annotation row.

In [50]:
raw_annotations_df = pd.io.json.json_normalize(latest_df.annotations, record_path='value', meta='classification_id')
raw_annotations_df.tail(5)

,answers,choice,filters,classification_id
4524,{},NOTHINGHERE,{},87927318
4525,"{'HOWMANY': '1', 'WHATBEHAVIORSDOYOUSEE': ['STANDING'], 'ARETHEREANYYOUNGPRESENT': 'NO'}",BEAR,{},87927343
4526,"{'HOWMANY': '1', 'WHATBEHAVIORSDOYOUSEE': ['MOVING'], 'ARETHEREANYYOUNGPRESENT': 'NO'}",DEER,{},87927361
4527,"{'HOWMANY': '1', 'WHATBEHAVIORSDOYOUSEE': ['MOVING'], 'ARETHEREANYYOUNGPRESENT': 'NO'}",LARGEBIRD,{},87927425
4528,{},IMNOTSURE,{},87927435


Better, but I'd really like the dictionaries in the new `answers` column to be columns of their own. I need to apply the `json_normalize` function yet again.

In [51]:
answers_df = pd.io.json.json_normalize(annotations_df.answers.tolist())
answers_df.tail(5)

,ARETHEREANYYOUNGPRESENT,HOWMANY,WHATBEHAVIORSDOYOUSEE
4524,NaN,NaN,NaN
4525,NO,1,[STANDING]
4526,NO,1,[MOVING]
4527,NO,1,[MOVING]
4528,NaN,NaN,NaN


Even better. I'm OK with the lists in the `WHATBEHAVIORSDOYOUSEE` column for the moment. I can turn them into separate boolean indicator columns, if warranted.

Right now, I want the `raw_annotations_df` and `answers_df` combined into a single DataFrame. Since I know there is exactly one dictionary in the `answers` column per choice, I can merge the two DataFrames based on the simple interger index.

In [56]:
annotations_df = pd.merge(raw_annotations_df.drop('answers', axis=1), answers_df, left_index=True, right_index=True)
annotations_df.tail(5)

,choice,filters,classification_id,ARETHEREANYYOUNGPRESENT,HOWMANY,WHATBEHAVIORSDOYOUSEE
4524,NOTHINGHERE,{},87927318,NaN,NaN,NaN
4525,BEAR,{},87927343,NO,1,[STANDING]
4526,DEER,{},87927361,NO,1,[MOVING]
4527,LARGEBIRD,{},87927425,NO,1,[MOVING]
4528,IMNOTSURE,{},87927435,NaN,NaN,NaN


I noted above that the `HOWMANY` was stored as a string in the original JSON. I can't turn that column into integers if I want to preserve the NaNs for non-responses, but I can use floats.

In [95]:
annotations_df['HOWMANY'] = annotations_df.HOWMANY.astype(float)

I see only empty dictionaries in the `filters` column. Do I really need to keep it?

In [71]:
annotations_df.filters.apply(lambda x: len(x) == 0).all()

True

Nope.

With that in mind, I want to perform the following bits of cleanup before declaring victory.

* Rename the ALLCAPS columns
* Reorder the columns
* Drop the empty `filters` column
* Make the values in the `young_present` column proper booleans, preserving NaNs for missing data

In [96]:
final_annotations_df = annotations_df.rename(columns={
    'ARETHEREANYYOUNGPRESENT': 'young_present',
    'HOWMANY': 'how_many',
    'WHATBEHAVIORSDOYOUSEE': 'behaviors_seen'
}).reindex(columns=['classification_id', 'choice', 'how_many', 'behaviors_seen', 'young_present'])
final_annotations_df['young_present'] = final_annotations_df.young_present.apply(lambda x: x if pd.isna(x) else x == 'YES')

In [97]:
final_annotations_df.tail(5)

,classification_id,choice,how_many,behaviors_seen,young_present
4524,87927318,NOTHINGHERE,NaN,NaN,NaN
4525,87927343,BEAR,1.0,[STANDING],False
4526,87927361,DEER,1.0,[MOVING],False
4527,87927425,LARGEBIRD,1.0,[MOVING],False
4528,87927435,IMNOTSURE,NaN,NaN,NaN


Done. Now I can easily query the DataFrame to answer questions like, how many time did users believe young were present?

In [98]:
final_annotations_df.young_present.value_counts()

False    1626
True       69
Name: young_present, dtype: int64

What species of young did annotators believe they saw?

In [99]:
df = final_annotations_df.dropna()
df[df.young_present]['choice'].value_counts()

BEAR                        48
INSECTORSPIDER               5
DEER                         4
REDWOLF                      3
SMALLBIRD                    3
OPOSSUM                      2
RACCOON                      1
COYOTE                       1
LARGEBIRD                    1
FROGTOADOROTHERAMPHIBIAN     1
Name: choice, dtype: int64

How many times did people think they saw more than one red wolf together?

In [102]:
df = final_annotations_df
df[df.choice.str.contains('WOLF') & (df.how_many > 1)]

,classification_id,choice,how_many,behaviors_seen,young_present
1378,74012494,REDWOLF,6.0,[STANDING],NaN
2389,74441376,REDWOLF,4.0,[STANDING],True


## Join on classification ID

I mentioned at the top that I'd really love to see the images associated with each response. I know I can show images in my notebook with Jupyter. I know the datset has a subject image filename per row. And I know I can join my new annotation DataFrame with the original based on the classification ID.

In [111]:
subject_df = latest_df[['classification_id', 'subject_ids', 'subject_data']].copy()
# the apply() function here should probably be split up; it's messy!
subject_df['subject_data'] = subject_df.subject_data.apply(lambda x: list(json.loads(x).values())[0]['Filename'])

In [114]:
subject_annotation_df = pd.merge(subject_df, final_annotations_df, how='outer', left_on='classification_id', right_on='classification_id')
subject_annotation_df.sample(20)

,classification_id,subject_ids,subject_data,choice,how_many,behaviors_seen,young_present
2033,74315250,10710105,mfdc7605_29515370606_o.jpg,IMNOTSURE,NaN,NaN,NaN
2867,78454464,10710126,mfdc7681_28924143084_o.jpg,BEAR,1.0,[STANDING],False
1085,73854414,10710556,mfdc8982_35101386705_o.jpg,BEAR,1.0,"[RESTING, STANDING]",False
3759,85603981,10710430,mfdc8777_34291389543_o.jpg,IMNOTSURE,NaN,NaN,NaN
1791,74304576,10722936,MFDC3070.jpg,DEER,1.0,[MOVING],False
2874,78454506,10723031,MFDC4056.jpg,DEER,1.0,"[MOVING, STANDING]",False
407,69134173,10710584,mfdc9048_35101385805_o.jpg,BEAR,1.0,[MOVING],False
4003,85803769,10710319,mfdc8512_34291393593_o.jpg,NOTHINGHERE,NaN,NaN,NaN
3715,85553879,10710074,mfdc7579_29515372636_o.jpg,IMNOTSURE,NaN,NaN,NaN
3208,78456276,10710590,mfdc9076_35101385715_o.jpg,BEAR,1.0,[INTERACTING],False


The link I'm missing is how I might use the image filenames or subject ID to resolve an image stored on the Zooniverse site. I see that when I'm classifying images, the page points to images using URLs like https://panoptes-uploads.zooniverse.org/production/subject_location/4ff121d7-13f2-47b2-86ea-dfdfc92c953d.jpeg but I don't know how to lookup or formulate an image name like that.